In [1]:
!pip install torch torchdata
!pip install transformers datasets
!pip install evaluate rouge_score loralib peft
!pip install accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
from datasets import load_dataset
import transformers
import accelerate
import torch


print(transformers.__version__)
print(accelerate.__version__)

4.31.0
0.20.3


In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [5]:
indices = [10, 15]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(indices):
    print(dash_line)
    print('Training Data ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['train'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['train'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Training Data  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Could you do me a favor?
#Person2#: Sure. What is it?
#Person1#: Could you run over to the store? We need a few things.
#Person2#: All right. What do you want me to get?
#Person1#: Well, could you pick up some sugar?
#Person2#: Okay. How much?
#Person1#: A small bag. I guess we also need a few oranges.
#Person2#: How many?
#Person1#: Oh, let's see. . . About six.
#Person2#: Anything else?
#Person1#: Yes. We're out of milk.
#Person2#: Okay. How much do you want me to get? A gallon?
#Person1#: No. I think a half gallon will be enough.
#Person2#: Is that all?
#Person1#: I think so. Have you got all that?
#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.
#Person1#: Do you have enough money?
#Person2#: I think so.
#Pe

In [6]:
model_name='facebook/bart-large-cnn'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [8]:
sentence = "What do you know about Taj Mahal?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )
print(sentence_encoded)
print(dash_line)
print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

{'input_ids': tensor([[    0,  2264,   109,    47,   216,    59, 17335,  3634,   337,   116,
             2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
---------------------------------------------------------------------------------------------------
ENCODED SENTENCE:
tensor([    0,  2264,   109,    47,   216,    59, 17335,  3634,   337,   116,
            2])

DECODED SENTENCE:
What do you know about Taj Mahal?


In [10]:
for i, index in enumerate(indices):
    dialogue = dataset['train'][index]['dialogue']
    summary = dataset['train'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=200,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Could you do me a favor?
#Person2#: Sure. What is it?
#Person1#: Could you run over to the store? We need a few things.
#Person2#: All right. What do you want me to get?
#Person1#: Well, could you pick up some sugar?
#Person2#: Okay. How much?
#Person1#: A small bag. I guess we also need a few oranges.
#Person2#: How many?
#Person1#: Oh, let's see. . . About six.
#Person2#: Anything else?
#Person1#: Yes. We're out of milk.
#Person2#: Okay. How much do you want me to get? A gallon?
#Person1#: No. I think a half gallon will be enough.
#Person2#: Is that all?
#Person1#: I think so. Have you got all that?
#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.
#Person1#: Do you have enough money?
#Person2#: I think so.
#Person1#: 

# Prompt Engineering

## Zero Shot Inference

In [11]:
for i, index in enumerate(indices):
  dialogue=dataset['train'][index]['dialogue']
  summary=dataset['train'][index]['summary']
  prompt=f'''
Summarize the conversation:
{dialogue}

Summary:
  '''
  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=200,
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print("PROMPT:")
  print(dash_line)
  print(prompt)
  print(dash_line)
  print('BASELINE HUMAN SUMMARY:')
  print(summary)
  print(dash_line)
  print('GENERATED SUMMARY:')
  print(output)
  print()

---------------------------------------------------------------------------------------------------
PROMPT:
---------------------------------------------------------------------------------------------------

Summarize the conversation:
#Person1#: Could you do me a favor?
#Person2#: Sure. What is it?
#Person1#: Could you run over to the store? We need a few things.
#Person2#: All right. What do you want me to get?
#Person1#: Well, could you pick up some sugar?
#Person2#: Okay. How much?
#Person1#: A small bag. I guess we also need a few oranges.
#Person2#: How many?
#Person1#: Oh, let's see. . . About six.
#Person2#: Anything else?
#Person1#: Yes. We're out of milk.
#Person2#: Okay. How much do you want me to get? A gallon?
#Person1#: No. I think a half gallon will be enough.
#Person2#: Is that all?
#Person1#: I think so. Have you got all that?
#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.
#Person1#: Do you have enough money?
#Person2#: I think so.

In [12]:
for i, index in enumerate(indices):
  dialogue=dataset['train'][index]['dialogue']
  summary=dataset['train'][index]['summary']
  prompt=f'''
Dialogues:
{dialogue}

What is going on here:
  '''
  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          inputs['input_ids'],
          max_new_tokens=200,
      )[0],
      skip_special_tokens=True
  )
  print(dash_line)
  print("PROMPT:")
  print(dash_line)
  print(prompt)
  print(dash_line)
  print('BASELINE HUMAN SUMMARY:')
  print(summary)
  print(dash_line)
  print('GENERATED SUMMARY:')
  print(output)
  print()

---------------------------------------------------------------------------------------------------
PROMPT:
---------------------------------------------------------------------------------------------------

Dialogues:
#Person1#: Could you do me a favor?
#Person2#: Sure. What is it?
#Person1#: Could you run over to the store? We need a few things.
#Person2#: All right. What do you want me to get?
#Person1#: Well, could you pick up some sugar?
#Person2#: Okay. How much?
#Person1#: A small bag. I guess we also need a few oranges.
#Person2#: How many?
#Person1#: Oh, let's see. . . About six.
#Person2#: Anything else?
#Person1#: Yes. We're out of milk.
#Person2#: Okay. How much do you want me to get? A gallon?
#Person1#: No. I think a half gallon will be enough.
#Person2#: Is that all?
#Person1#: I think so. Have you got all that?
#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.
#Person1#: Do you have enough money?
#Person2#: I think so.
#Person1#: Thank

## One shot Inference

In [13]:
def make_prompt(given_indices,test_index):
  prompt=''
  for index in given_indices:
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    prompt+=f"""
Dialogue:
{dialogue}

What is going on here?
{summary}
"""
  dialogue=dataset['test'][test_index]['dialogue']
  prompt+=f"""
Dialogue:
{dialogue}

What is going on here?
"""
  return prompt


In [14]:
given_index=[10]
test_index=20
one_shot_prompt=make_prompt(given_index,test_index)
print(one_shot_prompt)


Dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

What is going on here?
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

Dialogue:
#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Per

In [16]:
summary=dataset['test'][test_index]['summary']
inputs=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# thinks #Person2# has chicken pox and warns #Person2# about the possible hazards but #Person2# thinks it will be fine.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
Brian thinks #Person1# looks great and charming. #Person2# thinks he is coming down with chicken pox. #person1# thinks #person2# is a biohazard. # person1 thinks # person2 is contagious and should go take an oatmeal bath. # Person2 thinks # Person1 looks very pretty.


# Fine tuning using PEFT

In [17]:
def trainable_model_params(model):
  trainable_params=0
  all_params=0
  for _,param in model.named_parameters():
    all_params+=param.numel()
    if param.requires_grad:
      trainable_params+=param.numel()
  return f"trainable params: {trainable_params}\nall params: {all_params}\npercentage of trainable params: {100*trainable_params/all_params}"
print(trainable_model_params(model))

trainable params: 406290432
all params: 406290432
percentage of trainable params: 100.0


## LoRA Model

In [18]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="lora_only",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [19]:
peft_model=get_peft_model(model,lora_config)
print(trainable_model_params(peft_model))

trainable params: 1253376
all params: 407470080
percentage of trainable params: 0.30759951749095293


In [20]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [21]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})


In [22]:
import time
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-6,
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1,
    do_train=True,
    do_eval=True,
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation'],
)

In [23]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,11.384400


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/vocab.json',
 './peft-dialogue-summary-checkpoint-local/merges.txt',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [24]:
peft_model = PeftModel.from_pretrained(model,
                                       './peft-dialogue-summary-checkpoint-local',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)
peft_model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): BartForConditionalGeneration(
      (model): BartModel(
        (shared): Embedding(50264, 1024, padding_idx=1)
        (encoder): BartEncoder(
          (embed_tokens): Embedding(50264, 1024, padding_idx=1)
          (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
          (layers): ModuleList(
            (0-11): 12 x BartEncoderLayer(
              (self_attn): BartAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(
                  in_features=1024, out_features=1024, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_feature

In [25]:
print(trainable_model_params(model))

trainable params: 73728
all params: 407470080
percentage of trainable params: 0.018094089264173704


In [27]:
test_index=30
dialogue=dataset['test'][test_index]['dialogue']
summary=dataset['test'][test_index]['summary']
prompt=f"""
Summarize the conversation:
{dialogue}

Summary:
"""
inputs=tokenizer(prompt, return_tensors='pt')
peft_output=tokenizer.decode(
    peft_model.generate(
        input_ids=inputs['input_ids'].cuda(),
        generation_config=GenerationConfig(max_new_tokens=200, temperatue=1, do_sample=True),
    )[0],
    skip_special_tokens=True
)
original_output=tokenizer.decode(
    model.generate(
        input_ids=inputs['input_ids'].cuda(),
        generation_config=GenerationConfig(max_new_tokens=200,temperature=1, do_sample=True),
    )[0],
    skip_special_tokens=True
)
print(prompt)
print(dash_line)
print('BASELINE HUMAN SUMMARY:')
print(summary)
print(dash_line)
print('ORIGINAL MODEL SUMMARY:')
print(original_output)
print(dash_line)
print('PEFT MODEL SUMMARY')
print(peft_output)


Summarize the conversation:
#Person1#: Where are you going for your trip?
#Person2#: I think Hebei is a good place.
#Person1#: But I heard the north of China are experiencing severe sandstorms!
#Person2#: Really?
#Person1#: Yes, it's said that Hebes was experiencing six degree strong winds.
#Person2#: How do these storms affect the people who live in these areas?
#Person1#: The report said the number of people with respiratory tract infections tended to rise after sandstorms. The sand gets into people's noses and throats and creates irritation.
#Person2#: It sounds that sandstorms are trouble for everybody!
#Person1#: You are quite right.

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# plans to have a trip in Hebei but #Person1# says there are sandstorms in there.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL SUMMARY:
The